Mnist数据集共有60000张图片，50000train+10000test，每张图片为24*24，展开为(60000,784)的矩阵，标签为0-9的数字，训练前需要将其转换为one-hot编码。

In [5]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import warnings
warnings.filterwarnings('ignore')

In [21]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
batch_size = 64    # batch_size: 批次大小，表示每次训练放入多少数据
n_batch = mnist.train.num_examples // batch_size    # 计算一个周期有多少个批次，但存在一个问题：如果不能整除，则有余数部分的数据永远训练不到
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])
## U训练一个784-10的简单网络
# w = tf.Variable(tf.random_normal([784, 10]))
# b = tf.Variable(tf.zeros(10))
w = tf.Variable(tf.truncated_normal([784,10]))
b = tf.Variable(tf.zeros([10])+0.01)
z = tf.matmul(x,w) + b
prediction = tf.nn.softmax(z)
loss = tf.losses.mean_squared_error(y, prediction)
# 将损失函数换为交叉熵损失函数
# loss = tf.losses.softmax_cross_entropy(y, prediction)
train = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
# 定义精确率
corrent_pred = tf.equal(tf.argmax(y,1), tf.argmax(prediction, 1))
accuracy = tf.reduce_mean(tf.cast(corrent_pred, tf.float32))
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    # epoch(周期)：所有数据训练一次就是一个周期
    for epoch in range(21):
        for batch in range(n_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train,feed_dict={x:batch_xs, y:batch_ys})
        # 输出精确率
        acc = sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels})
        print('Epoch {},test accuracy is {}'.format(epoch, acc))


Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Epoch 0,test accuracy is 0.3012000024318695
Epoch 1,test accuracy is 0.36250001192092896
Epoch 2,test accuracy is 0.3919000029563904
Epoch 3,test accuracy is 0.4221000075340271
Epoch 4,test accuracy is 0.4629000127315521
Epoch 5,test accuracy is 0.4921000003814697
Epoch 6,test accuracy is 0.5288000106811523
Epoch 7,test accuracy is 0.5644999742507935
Epoch 8,test accuracy is 0.595300018787384
Epoch 9,test accuracy is 0.6165000200271606
Epoch 10,test accuracy is 0.6322000026702881
Epoch 11,test accuracy is 0.6444000005722046
Epoch 12,test accuracy is 0.6534000039100647
Epoch 13,test accuracy is 0.660099983215332
Epoch 14,test accuracy is 0.6654000282287598
Epoch 15,test accuracy is 0.6692000031471252
Epoch 16,test accuracy is 0.6725000143051147
Epoch 17,test accuracy is 0.6758999824523926
Epoch 1

###### 在分类问题中多使用交叉熵损失函数，在回归问题中多使用二次损失函数。

①初始化是普通随机数得到的结果：  
w = tf.Variable(tf.random_normal([784,10]))  
b = tf,Variable(tf.zeros([10]))

    Epoch 0,test accuracy is 0.23549999296665192
    Epoch 1,test accuracy is 0.3125999867916107
    Epoch 2,test accuracy is 0.4077000021934509
    Epoch 3,test accuracy is 0.46799999475479126
    Epoch 4,test accuracy is 0.5109999775886536
    Epoch 5,test accuracy is 0.5436000227928162
    Epoch 6,test accuracy is 0.5698999762535095
    Epoch 7,test accuracy is 0.592199981212616
    Epoch 8,test accuracy is 0.6085000038146973
    Epoch 9,test accuracy is 0.6226000189781189
    Epoch 10,test accuracy is 0.6342999935150146
    Epoch 11,test accuracy is 0.6468999981880188
    Epoch 12,test accuracy is 0.6596999764442444
    Epoch 13,test accuracy is 0.6693000197410583
    Epoch 14,test accuracy is 0.6786999702453613
    Epoch 15,test accuracy is 0.6868000030517578
    Epoch 16,test accuracy is 0.6933000087738037
    Epoch 17,test accuracy is 0.7006000280380249
    Epoch 18,test accuracy is 0.7059999704360962
    Epoch 19,test accuracy is 0.7117999792098999
    Epoch 20,test accuracy is 0.7139999866485596

本文亮点：  
①tensorflow中计算准确率  

    # 文中为：
    corrent_pred = tf.equal(tf.argmax(y, 1), tf.argmax(prediction, 1))
    accuracy = tf.reduce_mean(tf.cast(corrent_pred, tf.float32))
    ……
    acc = sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels})
    
②损失函数：  
会发现交叉熵损失函数要好于二次损失函数  
二次损失函数：$\frac12\sum_i{(y_i-\hat{y}_i){^2}}$  
交叉熵损失函数：$-\sum_i[{y_iln\hat{y}_i+(1-y_i)ln(1-\hat{y}_i)}]$

    # 二次损失函数
    loss = tf.losses.mean_squared_error(y, prediction)
    # 交叉熵损失函
    loss = tf.losses.softmax_cross_entropy(y, prediction)
在处理分类问题时多使用交叉熵损失函数，在处理回归问题时多使用二次损失函数。  
③用mini-batch梯度下降法：  

    # 先划分批次
    from tensorflow.examples.tutorials.mnist import input_data
    mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
    batch_size = 64
    n_batch = mnist.train.num_examples // batch_size
    ……
    for epoch in range(21):    # epoch为周期，一个周期会训练所有的数据
        for batch in range(n_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train, feed_dict={x:batch_xs, y:batch_ys})

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
